<a href="https://colab.research.google.com/github/harenlin/IMDB-Sentiment-Analysis-Using-BERT-Fine-Tuning/blob/main/BERT_Fine_Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Package

In [ ]:
# Install the required package
!pip install bert-for-tf2

     |████████████████████████████████| 51kB 6.2MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=0f4bb612a328aa164a22ead1cc15f9ff6142bcadfcde68d6626cb49e5fdd0973
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=335debbbb19bbb3d664d04e54a2d34ed56e5ff189258e082f1b06cbbb6a3042a
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=3c764acd0b17d753618db9ee329472dc9a8ba432fee9f9591dc82ea59724060c
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [ ]:
# Import modules
import os
import bert
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tqdm import tqdm
import matplotlib.pyplot as plt

print("TensorFlow Version:",tf.__version__)
print("Hub version: ",hub.__version__)
pd.set_option('display.max_colwidth',1000)

TensorFlow Version: 2.5.0
Hub version:  0.12.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Laod In Data & Data Preprocessing

In [ ]:
# Download data set from this website first:
# https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

# Read the IMDB Dataset.csv into Pandas dataframe
df = pd.read_csv('/content/drive/My Drive/Tensorflow Course/IMDB Dataset.csv')

In [ ]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [ ]:
# Take a peek at the dataset
df.head(5)

,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only ""has got all the polari"" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terribly well done.",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her ""sexy"" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than ""Devil Wears Prada"" and more interesting than ""Superman"" a great comedy to go see with friends.",positive
3,"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them.",negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the diffe

In [ ]:
print("The number of rows and columns in the dataset is: {}".format(df.shape))

The number of rows and columns in the dataset is: (50000, 2)


In [ ]:
# Identify missing values
df.apply(lambda x: sum(x.isnull()), axis=0)

review       0
sentiment    0
dtype: int64

In [ ]:
# Check the target class balance
df["sentiment"].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [ ]:
MAX_SEQ_LEN = 0
seq_len = []
for sent in df["review"]:
    if len(sent) > MAX_SEQ_LEN: MAX_SEQ_LEN = len(sent)
    seq_len.append(len(sent))
print(MAX_SEQ_LEN)
print(sum(seq_len)/len(df))

13704
1309.43102



Tokenizer Setup

In [ ]:
# Example of BERT Tokenizer Setup
# https://www.tensorflow.org/official_models/fine_tuning_bert
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file = os.path.join(gs_folder_bert, "vocab.txt"), do_lower_case=True)
print("Vocab size:", len(tokenizer.vocab))
tokens = tokenizer.tokenize("Hello TensorFlow!")
print(tokens)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)
print(tokenizer.convert_tokens_to_ids(['[CLS]', '[SEP]']))

Vocab size: 30522
['hello', 'tensor', '##flow', '!']
[7592, 23435, 12314, 999]
[101, 102]


In [ ]:
def create_tonkenizer(bert_layer):
    """Instantiate Tokenizer with vocab"""
    vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    do_lower_case = bert_layer.resolved_object.do_lower_case.numpy() 
    tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)
    print("Vocab size:", len(tokenizer.vocab))
    return tokenizer

Make Input Embeddings Suitable for BERT

In [ ]:
def get_ids(tokens, tokenizer, MAX_SEQ_LEN):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (MAX_SEQ_LEN - len(token_ids))
    return input_ids
# checkfunality of get_ids
# print(get_ids(tokenizer.tokenize("Hello TensorFlow!"), tokenizer, 500))

In [ ]:
def get_masks(tokens, MAX_SEQ_LEN):
    """Masks: 1 for real tokens and 0 for paddings"""
    return [1] * len(tokens) + [0] * (MAX_SEQ_LEN - len(tokens))

# # checkfunality of get_masks
# print(get_masks(tokenizer.tokenize("Hello TensorFlow!"), 500))  

In [ ]:
def get_segments(tokens, MAX_SEQ_LEN):
    """Segments: 0 for the first sequence, 1 for the second"""  
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (MAX_SEQ_LEN - len(tokens))

# checkfunality of get_masks
# print(get_segments(tokenizer.tokenize("Hello TensorFlow!"), 500))  

In [ ]:
def create_single_input(sentence, tokenizer, max_len):
    """Create an input from a sentence"""
    stokens = tokenizer.tokenize(sentence)
    stokens = stokens[:max_len] # max_len = MAX_SEQ_LEN - 2, why -2 ? ans: reserved for [CLS] & [SEP]
    stokens = ["[CLS]"] + stokens + ["[SEP]"]
    return get_ids(stokens, tokenizer, max_len+2), get_masks(stokens, max_len+2), get_segments(stokens, max_len+2)

# checkfunality of create_single_input
# res1, res2, res3 = create_single_input("Hello TensorFlow!", tokenizer, 500)
# print(res1)
# print(res2)
# print(res3)

In [ ]:
def convert_sentences_to_features(sentences, tokenizer, MAX_SEQ_LEN):
    """Convert sentences to features: input_ids, input_masks and input_segments"""
    input_ids, input_masks, input_segments = [], [], []
    for sentence in tqdm(sentences, position=0, leave=True):
      ids, masks, segments = create_single_input(sentence, tokenizer, MAX_SEQ_LEN-2) # why -2 ? ans: reserved for [CLS] & [SEP]
      # assert len(ids) == MAX_SEQ_LEN
      # assert len(masks) == MAX_SEQ_LEN
      # assert len(segments) == MAX_SEQ_LEN
      input_ids.append(ids)
      input_masks.append(masks)
      input_segments.append(segments)
    return [np.asarray(input_ids, dtype=np.int32), np.asarray(input_masks, dtype=np.int32), np.asarray(input_segments, dtype=np.int32)]

# checkfunality of convert_sentences_to_features
# [res1, res2, res3] = convert_sentences_to_features(["Hello TensorFlow!"], tokenizer, 500)
# print(res1, res2, res3)

# Define Model Architecture

In [ ]:
# please refer to the website: https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1
# to check the input and output of bert layer
# two outputs: a pooled_output of shape [batch_size, 768] with representations for the entire input sequences 
# and a sequence_output of shape [batch_size, max_seq_length, 768] with representations for each input token (in context).

def nlp_model(bert_base):
    # Load the pre-trained BERT base model
    bert_layer = hub.KerasLayer(handle=bert_base, trainable=True)  
    # BERT layer three inputs: ids, masks and segments
    input_ids = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_ids")           
    input_masks = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="input_masks")       
    input_segments = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32, name="segment_ids")

    inputs = [input_ids, input_masks, input_segments] # BERT inputs
    pooled_output, sequence_output = bert_layer(inputs) # BERT outputs
    
    x = Dense(units=768, activation='relu')(pooled_output) # hidden layer 
    x = Dropout(0.15)(x) 
    outputs = Dense(2, activation="softmax")(x) # output layer

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Model Training

Method 1

In [ ]:
# hyper-parameters
BATCH_SIZE = 8
EPOCHS = 1
MAX_SEQ_LEN = 500

# model construction (we construct model first inorder to use bert_layer's tokenizer)
bert_base = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
model = nlp_model(bert_base) 
model.summary()

# Create examples for training and testing
df = df.sample(frac=1) # Shuffle the dataset
# we would like to use bert tokenizer; therefore, chech model.summary() and find the index of bert_layer
tokenizer = create_tonkenizer(model.layers[3])

# gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
# tf.io.gfile.listdir(gs_folder_bert)
# tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file = os.path.join(gs_folder_bert, "vocab.txt"), do_lower_case=True)

# create training data and testing data
x_train = convert_sentences_to_features(df['review'][:40000], tokenizer, MAX_SEQ_LEN)
x_valid = convert_sentences_to_features(df['review'][40000:45000], tokenizer, MAX_SEQ_LEN)
x_test = convert_sentences_to_features(df['review'][45000:], tokenizer, MAX_SEQ_LEN)
df['sentiment'].replace('positive', 1., inplace=True)
df['sentiment'].replace('negative', 0., inplace=True)
one_hot_encoded = to_categorical(df['sentiment'].values)
y_train = one_hot_encoded[:40000]
y_valid = one_hot_encoded[40000:45000]
y_test =  one_hot_encoded[45000:]

Method 2

In [ ]:
# create tokenizer
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/v3/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file = os.path.join(gs_folder_bert, "vocab.txt"), do_lower_case=True)

# create training data and testing data
df = df.sample(frac=1) # Shuffle the dataset
x_train = convert_sentences_to_features(df['review'][:40000], tokenizer, MAX_SEQ_LEN)
x_valid = convert_sentences_to_features(df['review'][40000:45000], tokenizer, MAX_SEQ_LEN)
x_test = convert_sentences_to_features(df['review'][45000:], tokenizer, MAX_SEQ_LEN)
df['sentiment'].replace('positive', 1., inplace=True)
df['sentiment'].replace('negative', 0., inplace=True)
one_hot_encoded = to_categorical(df['sentiment'].values)
y_train = one_hot_encoded[:40000]
y_valid = one_hot_encoded[40000:45000]
y_test =  one_hot_encoded[45000:]

# hyper-parameters
BATCH_SIZE = 8
EPOCHS = 1
MAX_SEQ_LEN = 500

# model construction
bert_base = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1"
model = nlp_model(bert_base) 
model.summary()

100%|██████████| 5000/5000 [00:20<00:00, 247.14it/s]


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 500)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 500)]        0                                            
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      [(None, 768), (None, 109482241   input_ids[0][0]                  
                                                                 input_masks[0][0]          

In [ ]:
# use adam optimizer to minimize the categorical_crossentropy loss
optimizer = Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# fit the data to the model
history = model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

# save the trained model
model.save('/content/drive/My Drive/Tensorflow Course/imdb_bert_fine_tune.h5')

5000/5000 [==============================] - 4938s 984ms/step - loss: 0.2055 - accuracy: 0.9190 - val_loss: 0.1647 - val_accuracy: 0.9380


# Analyze Model Performence

In [ ]:
# load the pretrained nlp_model
from tensorflow.keras.models import load_model
imdb_model = load_model('/content/drive/My Drive/Tensorflow Course/imdb_bert_fine_tune.h5', custom_objects={'KerasLayer': hub.KerasLayer})

# predict on test dataset
from sklearn.metrics import classification_report
y_pred = np.argmax(imdb_model.predict(x_test), axis=1)

In [ ]:
print(classification_report(np.argmax(y_test, axis=1), y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      2534
           1       0.94      0.95      0.94      2466

    accuracy                           0.95      5000
   macro avg       0.95      0.95      0.95      5000
weighted avg       0.95      0.95      0.95      5000



In [ ]:
y_pred[:10]

array([0, 0, 0, 1, 1, 0, 0, 1, 0, 1])

In [ ]:
df['review'][45002]

'What can I say? I ignored the reviews and went to see it myself. Damn the reviews were so right. What a waste of money considering it\'s budget.<br /><br />Good thing, I went to see Kill Bill after this one.<br /><br />To see a really scary movie, would be Crossroads!<br /><br />Bottom line-- I like "Girl in Gold Boots" better than this crap.'

In [ ]:
df['review'][45003]

'A pretty transparent attempt to wring cash out of the thriving British club scene, Sorted is a film that shows promise in certain departments, but does very little else. A perfunctory thriller plot (which is there merely to string the club sequences together), variable acting and a pretty ludicrous script, all stop Sorted from being the showcase that director Jovy obviously intended.<br /><br />However, although Jovy is sometimes over indulgent (especially when using the often ill-fitting dance music) he does show potential, and the lack of an anti drugs message is enormously refreshing. Overall however, the film is a wasted opportunity, and the prospects for a great clubbing movie remain out there somewhere. Watchable nevertheless.'